In [17]:
import pandas as pd
import tensorflow as tf
tf.reset_default_graph()
import numpy as np
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [74]:
df = pd.read_csv('./loan.csv')


C:\Anaconda3\envs\tensor\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [75]:
df.term.replace(to_replace=[' 36 months', ' 60 months'],value=[1,0],inplace=True)
df.grade.replace(to_replace=['B', 'C', 'A', 'E', 'F', 'D', 'G'],value=[2,3,1,5,6,4,7],inplace=True)
positive = ['Fully Paid']

negative = ['Charged Off']

# filter out any word that is not within positive & negative
##"Does not meet the credit policy. Status:1"
##"Does not meet the credit policy. Status:0" are filtered out above

df = df[df['loan_status'].isin(positive + negative)].copy()

df['loan_status'] = df['loan_status'].isin(positive).astype(int) #.astype(int) will print "True" as "1"
df.emp_length.replace(to_replace=['10+ years', '< 1 year', '3 years', '9 years', '4 years', '5 years',
       '1 year', '6 years', '2 years', '7 years', '8 years', 'n/a'],value=[10,0,1,3,9,4,5,1,6,2,7,8]
                      ,inplace=True
                     )
positive =['OWN','MORTGAGE']
negative = ['RENT','NONE']
df = df[df.home_ownership.isin(positive+negative).copy()]
df['home_ownership'] = df['home_ownership'].isin(positive).astype(int)
v = ['Verified', 'Source Verified']
nv = ['Not Verified']
df = df[df.verification_status.isin(v+nv).copy()]
df.verification_status= df.verification_status.isin(v).astype(int)
df['addr_state'] = pd.factorize(df.addr_state)[0] + 1
df.initial_list_status.replace(to_replace=['w','f'],value=[0,1],inplace=True)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

newdf = df.select_dtypes(include=numerics)
newdf= newdf.dropna(axis=1)
y = newdf['loan_status']
del newdf['loan_status']
del newdf['id']
del newdf['member_id']
X=newdf

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [135]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [136]:
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

In [137]:
from tensorflow.contrib.keras import layers,models,losses,optimizers,metrics,activations

In [138]:
model = models.Sequential()

In [139]:
# model.add(layers.InputLayer(input_shape=[34]))
model.add(layers.Dense(units=32,activation='relu',input_dim=32))
model.add(layers.Dense(units=32,activation='relu'))
model.add(layers.Dense(units=2,activation='softmax'))

In [150]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [141]:
model.fit(scaled_x_train,y_train,epochs=10)

Epoch 1/10
176980/176980 [==============================] - 7s - loss: 0.0385 - acc: 0.9872       ETA: 23s - loss: 0 - ETA: 2s - loss: 0.0512 - acc: 0 - ETA: 2s - loss: 0.04 - ETA: 1s - loss: 0.0448 - ETA: 1s - loss: 
Epoch 2/10
176980/176980 [==============================] - 7s - loss: 0.0140 - acc: 0.9968     
Epoch 3/10
176980/176980 [==============================] - 6s - loss: 0.0121 - acc: 0.9974     
Epoch 4/10
176980/176980 [==============================] - 6s - loss: 0.0113 - acc: 0.9976     - ETA: 0s - loss: 0.0113
Epoch 5/10
176980/176980 [==============================] - 6s - loss: 0.0104 - acc: 0.9979     - ETA: 2s - loss:  - ETA: 1s - loss: 0.0 - ETA: 0s - loss: 0.0103 -
Epoch 6/10
176980/176980 [==============================] - 7s - loss: 0.0099 - acc: 0.9981     
Epoch 7/10
176980/176980 [==============================] - 6s - loss: 0.0093 - acc: 0.9981     - ETA: 3s - loss: 0. - ETA: 0s - loss: 0.00
Epoch 8/10
176980/176980 [==============================] - 6s - l

In [146]:
y_pred = model.predict_classes(scaled_x_test)

74240/75849 [============================>.] - ETA: 0s

In [147]:
from sklearn.metrics import classification_report

In [148]:
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      0.99      1.00     13588
          1       1.00      1.00      1.00     62261

avg / total       1.00      1.00      1.00     75849

